# Gradient Descent

Before running, you will need to download the file `image_theta.npz` from the dropbox. It is under the director `offset seeds and gifs`. This is where we will upload everything. Once downloaded, replace `DATA` with the correct path to `image_theta.npz`. You should be able to run all. This will run gradient descent out-of-box and will create a gif. Use this to see if your assigned data needs to be clicked through. If it doesn't, great! Save the npz and gifs to the dropbox. If it does, go to `click_to_angle.ipynb` and follow the instructions.

## 1. Functions

In [ ]:
import os
import numpy as np
import torch
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from scipy.ndimage import gaussian_filter
# from medpy.filter.smoothing import anisotropic_diffusion

In [ ]:
# Define global parameters
system_symmetry = 6.
k = 8.4
MS = torch.arange(2*system_symmetry)
ANGLES = torch.arange(0, system_symmetry)* 2 * torch.pi / system_symmetry
MAX_ITER_OFFSET = 51
LR = 1e-2
OFFSET_ADJUSTMENT = int(360/system_symmetry)
OFFSET_ADJUSTMENT_rad = np.deg2rad(OFFSET_ADJUSTMENT)
DATA = np.load("/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/image_theta.npz")["data"]
# DATA = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/npz_temp_sweep/image_theta.npz')['data']
DATA_THETA = torch.atan2(torch.tensor(DATA[1]), torch.tensor(DATA[0]))

# file_path = '/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/npz_sept_data/npz_field_sweep/neg29mT_553_50mW.npz'
# data = np.load(file_path)['data']
print(f'n_folds = {system_symmetry}')
print(f'k = {k}')


In [ ]:
# Function 1: mask and blur images we got from SANS_to_npz.ipynb
def mask_and_blur_images(array):
    """Applies a circular mask and Gaussian blur to intensity data."""
    x, y = np.meshgrid(np.arange(128), np.arange(128))
    radius = np.sqrt((x - 62)**2 + (y - 62)**2)
    mask = (radius > 18) & (radius < 30)
    for i in range(len(array)):
        masked_data = array[i] * mask
        array[i] = gaussian_filter(masked_data, sigma=0.65)
    return array

n_folds=6
def filter_function(k, theta, n_folds=n_folds):
    filter = torch.exp(k * torch.log((torch.cos(n_folds / 2 * theta))**2))
    # plt.imshow(filter)
    # plt.title(f'n_folds={n_folds}, k={k}')
    # plt.show()
    return filter

In [ ]:
# file_path = "/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/sep_data/"
file_path = '/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/'
file_path = '/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/npz_sept_numor_data/'
numor='56059'
movies = numor + '.npz'
# Define the movie you want to run GD and GS on as gif (gif = movies[i])
gif = movies
print(gif)
movie = np.load(file_path + gif)
intensity_data = torch.tensor(mask_and_blur_images(movie["data"]))
filename, _ = os.path.splitext(gif)
# print(filename)

print("Tensor shape should be (X,128,128), where X is the number of images.")
print(intensity_data.shape)

In [ ]:
# Function 2: get correct period indices
def get_correct_period_indices(loss_values):
    """Finds the correct period indices based on the maximum value in the loss values.
    If only one max index is found, adds adjacent indices at -60 or +60."""
    indices = [i for i, loss in enumerate(loss_values) if loss == max(loss_values)]
    if len(indices) > 1:
        return sorted(indices[:2])
    index = indices[0]
    adjacent_indices = [i for i in [index + OFFSET_ADJUSTMENT, index - OFFSET_ADJUSTMENT] if 0 <= i < len(loss_values)]
    return sorted([index] + adjacent_indices)

In [ ]:
# Function 3: Discrete search for number of domains and get approximate offset positions
def compute_closest_periodic_loss(intensity, prev_offset):
    """Compute the loss, first derivative, and second derivative for a range of offsets
    around the previous offset, selecting the closest periodic range."""
    
    # Define the periodic range and step size for sweeping through offset angles
    angle_period = np.deg2rad(360/system_symmetry)
    step_size_radians = np.deg2rad(1)
    offset_values = np.arange(prev_offset-angle_period, prev_offset+angle_period, step_size_radians)
    loss_values = []
    for offset in offset_values:
        evaluate_image_theta = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset)))**2))
        loss = -(intensity * evaluate_image_theta).sum()
        loss_values.append(loss.item())
    first_derivative = np.gradient(loss_values)
    second_derivative = np.gradient(first_derivative)
    selected_indices = get_correct_period_indices(loss_values)

    # Select the appropriate range of offsets, losses, and derivatives based on the period indices
    offset_values = np.array(offset_values[selected_indices[0]:selected_indices[1]+1])
    loss_values = np.array(loss_values[selected_indices[0]:selected_indices[1]+1])
    first_derivative = np.array(first_derivative[selected_indices[0]:selected_indices[1]+1])
    second_derivative = np.array(second_derivative[selected_indices[0]:selected_indices[1]+1])

    return offset_values, loss_values, first_derivative, second_derivative

In [ ]:
# Function 4: Get approximate offset positions through finding local maxima in second derivatives
def get_approx_offset_values(offset_values, second_derivative):
    """Identify approximate offset positions by finding local maxima and inflection points in the second derivative."""
    local_maxima_indices = argrelextrema(second_derivative, np.greater)[0]
    # Remove indices close to the edges because these local maximas might be introduced by the local maxima from the phase change
    filtered_indices = local_maxima_indices[(local_maxima_indices >= 5) & (local_maxima_indices <= len(second_derivative) - 6)]
    indices_info = [(index, offset_values[index], second_derivative[index]) for index in filtered_indices]
    # Sort by second derivative values (descending) and return top two offsets
    sorted_maxima = sorted(indices_info, key=lambda x: x[2], reverse=True)
    approx_offset_values = [offset for _, offset, _ in sorted_maxima][:2]
    return approx_offset_values

In [ ]:
# Function 5: Get approximate offset value for the other offset
def get_approximate_offset2(current_offset, approx_offset_values, reference_offset):
    """Calculate an approximate second offset based on provided offsets and a reference offset."""
    # Get approximate offset value for second offset
    offset_range = (reference_offset - OFFSET_ADJUSTMENT_rad/2, reference_offset + OFFSET_ADJUSTMENT_rad/2 + 0.1)
    # Edit the values (+- 60 deg.) so that we know which is the first offset
    adjusted_offsets = [
        offset + (OFFSET_ADJUSTMENT_rad if offset < offset_range[0] else -OFFSET_ADJUSTMENT_rad) * 
        int(not (offset_range[0] <= offset <= offset_range[1]))
        for offset in approx_offset_values
    ]
    # Calculate the differences to determine the second offset
    closest_offset = min(adjusted_offsets, key=lambda x: abs(x - reference_offset))
    adjusted_offsets = [offset for offset in adjusted_offsets if offset != closest_offset]
    return adjusted_offsets[0]

In [ ]:
# Function 6: helper function to visualize loss plot and its derivatives
def visualize_loss_plot_in_degrees(offset_values, loss_values, first_derivative, second_derivative):
    offset_values_deg = np.rad2deg(offset_values)
    fig, axes = plt.subplots(1,3, figsize=(15,5))
    axes[0].plot(offset_values_deg, loss_values)
    axes[0].set_xlabel("Offsets (degrees)")
    axes[0].set_title("Loss Values")
    axes[1].plot(offset_values_deg, first_derivative)
    axes[1].set_xlabel("Offsets (degrees)")
    axes[1].set_title("First Derivative")
    axes[2].plot(offset_values_deg, second_derivative)
    axes[2].set_xlabel("Offsets (degrees)")
    axes[2].set_title("Second Derivative")
    plt.show()

In [ ]:
# Function 7: helper function to normalize data
def normalize_min_max(data):
    array = data.detach().numpy() if isinstance(data, torch.Tensor) else np.array(data)
    norm_array = (array - array.min()) / (array.max() - array.min())
    return torch.tensor(norm_array) if isinstance(data, torch.Tensor) else norm_array

In [ ]:
# Function 8: helper function to fix snapback (assume offsets are in degree)
def adjust_offset_within_bounds(offset_list, angle_above_offset=50):
    angle_below_offset = OFFSET_ADJUSTMENT - angle_above_offset
    adjusted_offsets = []
    prev_offset = offset_list[0]
    for index, offset in enumerate(offset_list):
        if index == 0:
            adjusted_offsets.append(offset)
            prev_offset = offset
        else:
            offset_range = (prev_offset - angle_below_offset, prev_offset + angle_above_offset)
            while not (offset_range[0] <= offset <= offset_range[1]):
                offset += OFFSET_ADJUSTMENT if offset < offset_range[0] else -OFFSET_ADJUSTMENT
            adjusted_offsets.append(offset)
            prev_offset = offset
    return adjusted_offsets

In [ ]:
# Function 9: helper function to compute uncertainty
def compute_uncertainty(intensity, computed_offset):
    """Compute uncertainty using curvature: sigma = 1/sqrt(|H|)."""
    angle_period = np.deg2rad(10)
    step_size_radians = np.deg2rad(1)
    offset_values = np.arange(computed_offset.item()-angle_period, computed_offset.item()+angle_period, step_size_radians)
    loss_values = []
    for offset in offset_values:
        evaluate_image_theta = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset)))**2))
        loss = -(intensity * evaluate_image_theta).sum()
        loss_values.append(loss.item())
    first_derivative = np.gradient(loss_values)
    second_derivative = np.gradient(first_derivative)
    uncertainty = 1.0 / np.sqrt(np.abs(second_derivative[10])) / np.pi * 180
    return uncertainty

In [ ]:

def penalty_function(offset, penalty_weight=1, tolerance=-5):
    '''
    if movie has negative offsets, tolerance should be negative,
    vice versa for positive offsets.

    input tolerance in degrees
    input offset is a torch.tensor in radians
    '''
    offset = np.rad2deg(offset.clone().detach().numpy())
    penalty = penalty_weight*np.exp(offset+tolerance)
    penalty = torch.tensor(penalty)
    return penalty

# print('penalty = ', penalty_function(np.pi/24).item())



In [ ]:
# Function 10:

def gradient_descent_optimize_offset(intensity, offset1, offset2, k):
    """
    offset1 has the greatest loss score. 
    """
    
    # Store previous offsets
    prev_offset1 = offset1.item()
    prev_offset2 = offset2.item()
    
    # Get approximate offsets
    offset_range, loss_values, first_derivative, second_derivative = compute_closest_periodic_loss(intensity, prev_offset1)
    approx_offset_values = get_approx_offset_values(offset_range, second_derivative)
    number_of_domains = len(approx_offset_values)
    # visualize_loss_plot_in_degrees(offset_range, loss_values, first_derivative, second_derivative)
    offset_range2, _, _, second_derivative2 = compute_closest_periodic_loss(intensity, prev_offset2)
    approx_offset_values2 = get_approx_offset_values(offset_range2, second_derivative2)
    # print("Number of domains = ", number_of_domains, ", Offset values are approx. ", approx_offset_values, approx_offset_values2)
    
    # Take care of errorous data
    if number_of_domains == 0:
        return offset1, offset2, intensity, intensity, 0, 0, 0, 0

    # GD1
    opt = torch.optim.Adam([offset1], lr=LR)
    for i in range(MAX_ITER_OFFSET):
        evaluate_image_theta = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset1)))**2)) # sin(3x)^{2k}
        intensity = normalize_min_max(intensity.clone().detach())
        # Only penalize if offset1 is less than -tolerance
        penalty = penalty_function(offset1)
        loss = -(intensity * evaluate_image_theta).sum() + penalty
        opt.zero_grad()
        loss.backward()
        opt.step()

    if len(approx_offset_values) > 1:
        offset1_tmp_value = get_approximate_offset2(prev_offset1, approx_offset_values, offset1.item())
        offset1_tmp = torch.tensor(offset1_tmp_value, requires_grad=True)
        opt_tmp = torch.optim.Adam([offset1_tmp], lr=LR)
        for i in range(10):
            evaluate_image_theta_tmp = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset1_tmp)))**2))
            loss_tmp = -(intensity * evaluate_image_theta_tmp).sum()
            opt_tmp.zero_grad()
            loss_tmp.backward()
            opt_tmp.step()
        if loss_tmp.item() <= loss.item():
            # print(offset1_tmp, offset1, "offset1 switched!")
            loss = loss_tmp
            offset1 = offset1_tmp
            evaluate_image_theta = evaluate_image_theta_tmp

    # GD2
    opt2 = torch.optim.Adam([offset2], lr=LR)
    for i in range(MAX_ITER_OFFSET):
        evaluate_image_theta2 = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset2)))**2))
        loss2 = -(intensity * evaluate_image_theta2).sum()
        opt2.zero_grad()
        loss2.backward()
        opt2.step()

    if len(approx_offset_values2) > 1:
        offset2_tmp_value = get_approximate_offset2(prev_offset2, approx_offset_values2, offset2.item())
        offset2_tmp = torch.tensor(offset2_tmp_value, requires_grad=True)
        opt2_tmp = torch.optim.Adam([offset2_tmp], lr=LR)
        for i in range(10):
            evaluate_image_theta2_tmp = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset2_tmp)))**2))
            loss2_tmp = -(intensity * evaluate_image_theta2_tmp).sum()
            opt2_tmp.zero_grad()
            loss2_tmp.backward()
            opt2_tmp.step()
        if loss2_tmp.item() >= loss2.item():
            # print(offset2_tmp, offset2, "offset2 switched!")
            loss2 = loss2_tmp
            offset2 = offset2_tmp
            evaluate_image_theta2 = evaluate_image_theta2_tmp

    uncertainty1 = compute_uncertainty(intensity, offset1)
    uncertainty2 = compute_uncertainty(intensity, offset2)
    # print(penalty)
    return offset1, offset2, evaluate_image_theta, evaluate_image_theta2, loss.item(), loss2.item(), uncertainty1, uncertainty2












# Function 10:
def flipped_gradient_descent_optimize_offset(intensity, offset1, offset2, k):
    """
    offset2 has the greatest loss score
    """
    
    # Store previous offsets
    prev_offset1 = offset1.item()
    prev_offset2 = offset2.item()
    
    # Get approximate offsets
    offset_range, loss_values, first_derivative, second_derivative = compute_closest_periodic_loss(intensity, prev_offset1)
    approx_offset_values = get_approx_offset_values(offset_range, second_derivative)
    number_of_domains = len(approx_offset_values)
    # visualize_loss_plot_in_degrees(offset_range, loss_values, first_derivative, second_derivative)
    offset_range2, _, _, second_derivative2 = compute_closest_periodic_loss(intensity, prev_offset2)
    approx_offset_values2 = get_approx_offset_values(offset_range2, second_derivative2)
    # print("Number of domains = ", number_of_domains, ", Offset values are approx. ", approx_offset_values, approx_offset_values2)
    
    # Take care of errorous data
    if number_of_domains == 0:
        return offset1, offset2, intensity, intensity, 0, 0, 0, 0

    # GD1
    opt = torch.optim.Adam([offset1], lr=LR)
    for i in range(MAX_ITER_OFFSET):
        evaluate_image_theta = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset1)))**2)) # sin(3x)^{2k}
        intensity = normalize_min_max(intensity.clone().detach())
        # Only penalize if offset1 is less than -tolerance
        penalty = penalty_function(offset1)
        loss = -(intensity * evaluate_image_theta).sum() + penalty
        opt.zero_grad()
        loss.backward()
        opt.step()

    if len(approx_offset_values) > 1:
        offset1_tmp_value = get_approximate_offset2(prev_offset1, approx_offset_values, offset1.item())
        offset1_tmp = torch.tensor(offset1_tmp_value, requires_grad=True)
        opt_tmp = torch.optim.Adam([offset1_tmp], lr=LR)
        for i in range(10):
            evaluate_image_theta_tmp = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset1_tmp)))**2))
            loss_tmp = -(intensity * evaluate_image_theta_tmp).sum()
            opt_tmp.zero_grad()
            loss_tmp.backward()
            opt_tmp.step()
        if loss_tmp.item() >= loss.item():
            # print(offset1_tmp, offset1, "offset1 switched!")
            loss = loss_tmp
            offset1 = offset1_tmp
            evaluate_image_theta = evaluate_image_theta_tmp

    # GD2
    opt2 = torch.optim.Adam([offset2], lr=LR)
    for i in range(MAX_ITER_OFFSET):
        evaluate_image_theta2 = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset2)))**2))
        loss2 = -(intensity * evaluate_image_theta2).sum()
        opt2.zero_grad()
        loss2.backward()
        opt2.step()

    if len(approx_offset_values2) > 1:
        offset2_tmp_value = get_approximate_offset2(prev_offset2, approx_offset_values2, offset2.item())
        offset2_tmp = torch.tensor(offset2_tmp_value, requires_grad=True)
        opt2_tmp = torch.optim.Adam([offset2_tmp], lr=LR)
        for i in range(10):
            evaluate_image_theta2_tmp = torch.exp(k * torch.log((torch.sin((system_symmetry/2) * (DATA_THETA + offset2_tmp)))**2))
            loss2_tmp = -(intensity * evaluate_image_theta2_tmp).sum()
            opt2_tmp.zero_grad()
            loss2_tmp.backward()
            opt2_tmp.step()
        if loss2_tmp.item() <= loss2.item():
            # print(offset2_tmp, offset2, "offset2 switched!")
            loss2 = loss2_tmp
            offset2 = offset2_tmp
            evaluate_image_theta2 = evaluate_image_theta2_tmp

    uncertainty1 = compute_uncertainty(intensity, offset1)
    uncertainty2 = compute_uncertainty(intensity, offset2)
    # print(loss.item(), loss2.item())
    return offset1, offset2, evaluate_image_theta, evaluate_image_theta2, loss.item(), loss2.item(), uncertainty1, uncertainty2





## 2. Run gradient descent

In [ ]:
offset1 = torch.tensor(0., requires_grad=True)
offset2 = torch.tensor(0., requires_grad=True)
    offset1, offset2, image1, image2, loss1, loss2, u1, u2 = gradient_descent_optimize_offset(intensity_data[0], offset1, offset2, k)
    print(np.rad2deg(offset1.item()))
    plt.imshow(normalize_min_max(intensity_data[0]) + image1.detach().numpy())

In [ ]:
# Apply functions
# Initialize offsets
offset1 = torch.tensor(0., requires_grad=True)
offset2 = torch.tensor(0., requires_grad=True)
tolerance = torch.tensor(30) # in degrees

print(k, system_symmetry, numor, tolerance.item())

# Initialize lists
frame_ranges = np.arange(0, len(intensity_data), 1)
offset_list1, offset_list2 = [], []
image_list1, image_list2 = [], []
loss_list1, loss_list2 = [], []
uncertainty_list1, uncertainty_list2 = [], []

for i in frame_ranges:
    if i == 0:
        offset1, offset2, image1, image2, loss1, loss2, u1, u2 = gradient_descent_optimize_offset(intensity_data[i], offset1, offset2, k)
        offset_list1.append(offset1.item())
        offset_list2.append(offset2.item())
        image_list1.append(image1)
        image_list2.append(image2)
        loss_list1.append(loss1)
        loss_list2.append(loss2)
        uncertainty_list1.append(u1)
        uncertainty_list2.append(u2)
    else:
        # Run gradient descent for the current frame
        offset1_new, offset2_new, image1_new, image2_new, loss1_new, loss2_new, u1_new, u2_new = gradient_descent_optimize_offset(intensity_data[i], offset1, offset2, k)

        # Ensure there is a previous offset to compare against
        if len(offset_list1) > 0:
            previous_offset1 = torch.tensor(offset_list1[-1])
            previous_offset2 = torch.tensor(offset_list2[-1])

            # Check if offset1 moved in the opposite direction by more than tolerance
            if torch.rad2deg(offset1_new - previous_offset1) > tolerance or torch.rad2deg(offset2_new - previous_offset2) > tolerance or torch.abs(torch.rad2deg(offset1_new - previous_offset1)) > 25:
                print('------------')
                print(f'{i}: flip')
                print(round(torch.rad2deg(offset1_new - previous_offset1).item(), 3), '>', tolerance.item())
                    #    'or',round(torch.rad2deg(offset2_new - previous_offset2).item(),3), '>', tolerance.item() )

                # If so, rerun with the flipped function
                offset1_new, offset2_new, image1_new, image2_new, loss1_new, loss2_new, u1_new, u2_new = flipped_gradient_descent_optimize_offset(intensity_data[i], offset1, offset2, k)

        # Append results (store in radians)
        # print(offset1_new.item(),offset2_new.item())
        offset_list1.append(offset1_new.item())
        offset_list2.append(offset2_new.item())
        image_list1.append(image1_new)
        image_list2.append(image2_new)
        loss_list1.append(loss1_new)
        loss_list2.append(loss2_new)
        uncertainty_list1.append(u1_new)
        uncertainty_list2.append(u2_new)

        # Update offsets for next iteration
        offset1, offset2 = offset1_new, offset2_new



print("Done!")


## Plotting

In [ ]:
# Sava data

offset1cm = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_{numor}_offset1_seed.npz')['data']
offset2cm = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_{numor}_offset2_seed.npz')['data']
offset1yc = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/yc_{numor}_offset2_seed.npz')['data']
offset2yc = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/yc_{numor}_offset1_seed.npz')['data']
offset1dr = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/dr_{numor}_offset2_seed.npz')['data']
offset2dr = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/dr_{numor}_offset1_seed.npz')['data']
offset1nc = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/nc_{numor}_offset2_seed.npz')['data']
offset2nc = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/nc_{numor}_offset1_seed.npz')['data']
offset1oob = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/{numor}_offsets.npz')['offset1']
offset2oob = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/{numor}_offsets.npz')['offset2']
# (1) Visualize offsets (post-process if necessary)
# offset_list1_fix = adjust_offset_within_bounds(offset_list1, 35)
# offset_list2_fix = adjust_offset_within_bounds(offset_list2, 30)

colors = {
    "cm": "blue",
    "yc": "red",
    "dr": "green",
    "nc": "purple",
    "oob": 'orange',
    "error": 'hotpink'
}

plt.figure(figsize=(10, 10))
plt.plot(offset1oob-offset1oob[0], color=colors["oob"], label=f'offset1 oob {numor}', linestyle='dotted')
plt.plot(offset2oob-offset2oob[0], alpha=0.5, color=colors["oob"], label=f'offset2 oob {numor}', linestyle='-')
# plt.errorbar(np.arange(0, len(offset_list1)), np.rad2deg(offset_list1) - np.rad2deg(offset_list1)[0], yerr=uncertainty_list1, label=f"offset1 {numor} GDratchet", color=colors["error"], linestyle='dotted', capsize=2)
# plt.errorbar(np.arange(0, len(offset_list2)), np.rad2deg(offset_list2) - np.rad2deg(offset_list2)[0], yerr=uncertainty_list2, label=f"offset2 {numor} GDratchet", color=colors["error"], linestyle='-', capsize=2)
# plt.plot(np.rad2deg(offset_list1) - np.rad2deg(offset_list1)[0], label=f"offset1 {numor} GDratchet", color=colors["error"], linestyle='dotted')
# plt.plot(np.rad2deg(offset_list2) - np.rad2deg(offset_list2)[0], label=f"offset2 {numor} GDratchet", color=colors["error"], linestyle='-')

plt.plot(offset1cm-offset1cm[0], label=f'offset1 cm {numor}', color=colors['cm'], linestyle='dotted')
plt.plot(offset2cm-offset2cm[0], label=f'offset2 cm {numor}', color=colors['cm'], linestyle='-')
plt.plot(offset1yc-offset1yc[0], label=f'offset1 yc {numor}', color=colors["yc"], linestyle='dotted')
plt.plot(offset2yc-offset2yc[0], label=f'offset2 yc {numor}', color=colors["yc"], linestyle='-')
plt.plot(offset1dr-offset1dr[0], label=f'offset1 dr {numor}', color=colors["dr"], linestyle='dotted')
plt.plot(offset2dr-offset2dr[0], label=f'offset2 dr {numor}', color=colors["dr"], linestyle='-')
plt.plot(offset1nc-offset1nc[0], label=f'offset1 nc {numor}', color=colors["nc"], linestyle='dotted')
plt.plot(offset2nc-offset2nc[0], label=f'offset2 nc {numor}', color=colors["nc"], linestyle='-')


plt.xlabel("index")
plt.ylabel("offset")
plt.title(f"{numor}, iterations={MAX_ITER_OFFSET}, k={k}, tolerance={tolerance}deg")
plt.grid(True)
plt.legend()
plt.show()

# # (2) Visualize loss values
# plt.plot(np.array(loss_list1), label="Loss1")
# plt.plot(np.array(loss_list2), label="Loss2")
# plt.xlabel("Frames")
# plt.ylabel("Loss")
# plt.legend()
# # plt.show()

# np.savez(f"{filename}_offsets.npz", 
#          offset1=np.array(offset_list1), offset2=np.array(offset_list2), 
#          loss1=np.array(loss_list1),loss2=np.array(loss_list2), 
#          uncertainty1=np.array(uncertainty_list1), uncertainty2=np.array(uncertainty_list2))

## 3. Slider

In [ ]:
from matplotlib.widgets import Slider
from ipywidgets import interact, IntSlider

offsets1 = np.array(offset_list1)
# offsets1 = np.rad2deg(offsets1)

offsets2 = np.array(offset_list2)
# offsets2 = np.rad2deg(offsets2)
n_folds = 1
k=500
print("n_folds =", n_folds)
print('k value =', k)



def interactive_plot(frame_idx):
    '''Plot the intensity data and filter images with respect to the selected frame index.'''
    
    image1 = filter_function(k, DATA_THETA + offsets1[frame_idx])
    image2 = filter_function(k, DATA_THETA + offsets2[frame_idx])
    
    # Create figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    # Plot the first image (combined filter + intensity data)
    ax1.imshow(image1 + 2*image2 + normalize_min_max(intensity_data[frame_idx]), origin="lower")
    ax1.set_title(f"Frame {frame_idx+1}: Filter + Intensity")
    ax1.axis("off")
    
    # Plot the second image (raw intensity data)
    ax2.imshow(intensity_data[frame_idx], origin="lower")
    ax2.set_title(f"Frame {frame_idx+1}: Intensity Data")
    ax2.axis("off")
    
    # Update the figure title with offset values
    fig.suptitle(f"Time={(frame_idx+1)*10}s, Offset1={round(np.rad2deg(offsets1[frame_idx]), 3)}°, Offset2={round(np.rad2deg(offsets2[frame_idx]), 3)}°")
    
    # Display the plot
    plt.tight_layout()
    plt.show()

# Create interactive plot with sliders for frame index
interact(
    interactive_plot,
    frame_idx=IntSlider(value=0, min=0, max=len(intensity_data)-1, step=1, description='Frame Index')
);


## make gif

In [ ]:
# # Save gif
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

offsets1 = np.array(offset_list1)
offsets1 = np.deg2rad(offsets1)
offsets2 = np.array(offset_list2)
offsets2 = np.deg2rad(offsets2)



def update(i):
    image1 = filter_function(k, DATA_THETA+offsets1[i])
    image2 = filter_function(k, DATA_THETA+offsets2[i])
    for ax in axes:
        ax.clear()
    axes[0].imshow(image1 + image2 + normalize_min_max(intensity_data[i]), origin="lower")
    axes[0].set_title("GD")
    axes[0].axis("off")
    axes[1].imshow(intensity_data[i], origin="lower")
    axes[1].set_title("Intensity Data")
    axes[1].axis("off")
    fig.suptitle(f"Time={(i+1)*10}s, Offset1={round(offsets1[i],3)}deg, Offset2={round(offsets2[i],3)}deg")

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
# ani = FuncAnimation(fig, update, frames=len(intensity_data), interval=200)
# ani.save(f'{filename}_scrap.gif', writer="pillow")
# plt.show()